# LDA Topic Modelling

* This notebook is showcases the process of building an NLP Topic Model using `Latent Dirichlet Allocation` method. 
* The dataset we are going to use are `title` and `soft title` from `apify_dataset_clean.csv`. 

## Table Of Contents

## Installations


In [1]:
# ## installing required libraries
! pip install beautifulsoup4
! pip install pandas
! pip install numpy
! pip install plotly
! pip install nbformat
! pip install ipykernel
! pip install matplotlip
! pip install wordcloud
! pip install gensim
! pip install pyLDAvis
! pip install nltk
! pip install -U pip setuptools wheel
! pip install -U spacy
! python -m spacy download en_core_web_trf 


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


...

  Using cached pip-22.3.1-py3-none-any.whl (2.1 MB)
  Using cached setuptools-65.6.3-py3-none-any.whl (1.2 MB)
  Using cached wheel-0.38.4-py3-none-any.whl (36 kB)


ERROR: To modify pip, please run the following command:
D:\workspace\Aletheia\env\Scripts\python.exe -m pip install -U pip setuptools wheel

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 460.3/460.3 MB 5.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Imports

In [64]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import re
import string
from bs4 import BeautifulSoup
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
from pprint import pprint

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models.ldamulticore import LdaMulticore

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as io

# loading library
import pickle


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gaura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Reading Data

In [3]:
## reading manaully scrapped data
data = pd.read_csv('../data/apify_dataset_clean.csv')
print(data.shape)

(11003, 13)


## Utility Functions

### Preparing Stop Words

In [4]:
## extending stopwords
# lets break down the cleaning functions into smaller functions
stop_words = nltk.corpus.stopwords.words('english')

## trying to remove stopwords from stopwords super set. 
stopwords_super_set = pd.read_csv("../data/stopwords/sw1k.csv")

## filtering stopwords to pronouns and other type
stopwords_to_remove = list(stopwords_super_set.loc[(stopwords_super_set["type"] == "G" ), "term"])


# stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'say', 'one', 'time', 'people',
#                   'know', 'like', 'tell', 'get', 'year', 'go', 'around', 'award', 'actually', 'carry',
#                    'new', 'it', 'show', 'news', 'go', 'fox', 'make', 'do', 'not', 'say',
#                    'also', 'love', 'it', 'star', 'go', 'do', 'say', 'not', 'said'
#                    ])

# stop_words.extend(stopwords_to_remove)
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Common Utility Functions

In [5]:
nlp = spacy.load('en_core_web_trf')
# Utility Functions for Text Cleaning
def sent_to_words(sentences):
    for sentence in sentences:
        yield (simple_preprocess(str(sentence), deacc=True))

# function to clean html tags from text


def clean_html(html):
    # parse html content
    soup = BeautifulSoup(html, "html.parser")
    for data in soup(['style', 'script', 'code', 'a']):
        # Remove tags
        data.decompose()
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

# function to convert text to lowercase


def lower_case(text):
    return text.lower()

# function to remove line breaks


def remove_line_breaks(text):
    return re.sub(r'\n', '', text)

# function to remove punctuation


def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# function to remove numbers


def remove_numbers(text):
    return re.sub(r'\d+', '', text)

# function to remove extra spaces


def remove_extra_spaces(text):
    return re.sub(' +', ' ', text)

# function to remove stopwords


def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# helper function to create pos tags


def create_pos_tag(str_sent):
    return nlp(str_sent)

# function for text lemmatization using spac
##'ADJ', 'VERB'
def lemmatization(texts, allowed_postags=['PROPN', 'NOUN']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append(
            [token.lemma_ for token in doc if (token.pos_ in allowed_postags and token.is_alpha and token.is_stop == False)])
    return texts_out

def lemmatization_without_pos(texts):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append(
            [token.lemma_ for token in doc])
    return texts_out


def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]


def make_trigrams(texts, bigram_mod, trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

## helper function to create pos tags distribution
def create_pos_tags_distribution(docs = []):
    token_distribution = {}
    is_alpha = 0
    is_stop = 0
    for doc in docs:
        for token in doc:
            token_distribution[token.pos_] = token_distribution.get(token.pos_, 0) + 1
            if(token.is_alpha):
                is_alpha += 1
            if(token.is_stop):
                is_stop += 1
    return token_distribution, is_alpha, is_stop


# function to create n-grams from noun chunks
def create_noun_chunk_ngrams(docs):
    n_gram_docs = []
    for doc in docs:
        doc_text = doc.text
        for chunk in doc.noun_chunks:
            chunk_n_gram = "_".join(chunk.text.split(" "))
            doc_text = doc_text.replace(chunk.text, chunk_n_gram)
            # print(chunk_n_gram, chunk.root.text,
            #       chunk.root.dep_, chunk.root.head.text)
        n_gram_docs.append(doc_text.split(" "))
    return n_gram_docs


def lemmatization_noun_chunks(texts):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if (
            ("_" in token.text) or ## if the token is a noun chunk allow that
            (token.pos_ in ['NOUN', 'PROPN'] and token.is_alpha and token.is_stop == False) ## if the token is a noun or proper noun allow that
        )])
    return texts_out

### Gensim Models Utility Functions

In [8]:
## function to compute optimal parameters for LDA model
def compute_coherence_values(dictionary, corpus, id2word, texts, num_topics, passes, chunk_sizes=[200]):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    params = []
    for num_topic in num_topics:
        for chunk_size in chunk_sizes:
            for num_passes in passes:
                model = LdaModel(corpus=corpus,
                                 id2word=id2word,
                                 num_topics=num_topic,
                                 random_state=100,
                                 update_every=1,
                                 chunksize=chunk_size,
                                 passes=num_passes,
                                 per_word_topics=True)
                model_list.append(model)
                coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
                coherence_lda = coherencemodel.get_coherence()
                coherence_values.append(coherence_lda)
                params.append({'num_topics': num_topic, 'chunk_size': chunk_size, 'passes': num_passes})

    return model_list, coherence_values, params

def analyze_gensim_lda_model(lda_model, corpus, id2word, texts, num_topics, passes, chunk_sizes=[200]):
    # Compute Perplexity
    print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)

## helper functions to visualize LDA model
def visualize_gensim_lda_model(lda_model, corpus, id2word, filename="gensim_lda.html"):
    # Visualize the topics
    pyLDAvis.enable_notebook()
    vis = gensimvis.prepare(lda_model, corpus, id2word)
    vis.save(filename)

### Sklearn Model Utility Functions

In [79]:
import numpy as np


# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)


def print_sklearn_sparcity(data_vectorized):
    # Materialize the sparse data
    data_dense = data_vectorized.todense()

    # Compute Sparsicity = Percentage of Non-Zero cells
    print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")


def create_sklearn_dominent_topic_dataframe(lda_model, data_vectorized):
    lda_output = lda_model.transform(data_vectorized)
    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]
    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]
    
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic
    return df_document_topic

def print_sklearn_dominant_topics(lda_model, data_vectorized):
    df_document_topic = create_sklearn_dominent_topic_dataframe(lda_model, data_vectorized)
    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    return df_document_topics

def print_sklearn_topic_distribution(lda_model, data_vectorized):
    df_document_topic = create_sklearn_dominent_topic_dataframe(lda_model, data_vectorized)
    df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents").rename(columns={'index':'Topic'})
    # df_topic_distribution.columns = ["Topic Num", "Num Documents"]
    return df_topic_distribution


# Show top n keywords for each topic
def show_sklearn_topics(vectorizer, lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

def format_sklearn_topics(topic_keywords):
    df_topic_keywords = pd.DataFrame(topic_keywords)
    df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
    df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
    return df_topic_keywords

def analyze_sklearn_lda_model(lda_model, data_vectorized):
    # Log Likelyhood: Higher the better
    print("Log Likelihood: ", lda_model.score(data_vectorized))
    # Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
    print("Perplexity: ", lda_model.perplexity(data_vectorized))

## helper function to visualize lda model
def visualize_sklearn_lda_model(lda_model, data_vectorized, vectorizer, mds='tsne'):    
    pyLDAvis.enable_notebook()
    panel2 = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds=mds)
    return panel2

## EDA on Title Data

In [10]:
## quick and dirty test to see if we should use `title` or `soft_title` column. Using the longer column assuming it has more data. 
data[data['title'].str.len() > data["soft_title"].str.len()]


,url,author,date,title,soft_title,description,text,day,month,year,month_name,word_count,line_count


In [11]:
## check for duplicates
data["soft_title"].duplicated().sum()

3284

In [12]:
## there are 3287 duplicates in soft_title. But the URLs are different. But since we are using soft_title for topic modelling, we will remove duplicates

data.drop_duplicates(subset ="soft_title", inplace = True)

In [13]:
## sanity checking for duplicates
data["soft_title"].duplicated().sum()

0

In [14]:
data['title_word_count'] = data['soft_title'].apply(lambda x: len(str(x).split(" ")))

data['title_word_count'].describe()


count    7719.000000
mean       13.107397
std         2.846463
min         3.000000
25%        11.000000
50%        13.000000
75%        15.000000
max        23.000000
Name: title_word_count, dtype: float64

##### Notes
* Couple of interesting outliers, a title with 23 words and 4 words. 
* Lets check them out. 

In [15]:
## checkout outliers 
data[data["title_word_count"] > 15]["title"]

0        Biden says after Xi meeting he doesn’t believe...
3        Katie Hobbs defeats GOP challenger Kari Lake, ...
15       On this day in history, Nov. 15, 1956, Elvis m...
16       Walker hauls in $11 million since start of Geo...
20       Having sized up Biden, Putin invaded Ukraine. ...
                               ...                        
10982    Serena and Venus Williams lose what may be fin...
10986    Ye gets slammed by rapper Meek Mill after Tuck...
10988    Novak Djokovic points to 'positive signs' ahea...
10994    Erie County DA says domestic terrorism charges...
10996    Wimbledon fan takes legal action after Nick Ky...
Name: title, Length: 1607, dtype: object

In [16]:
## checking the distribution of word count in title
fig = px.histogram(data, x="title_word_count", title="Distribution of Word Count in Title")
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



##### Notes
* Word counts are fairly distributed. 

In [17]:
## lets create POS tags for each title and see the distribution of POS tags
# docs = [nlp(text) for text in data['soft_title']]
docs = data["soft_title"].apply(create_pos_tag)

In [18]:
## creating pos tags distribution
token_distribution, is_alpha, is_stop = create_pos_tags_distribution(docs)

In [19]:
## convert the dictionary to a dataframe
token_distribution_df = pd.DataFrame.from_dict(token_distribution, orient='index', columns=['count']).reset_index().rename(columns={"index": "tags"})


In [20]:
## lets create a distribution of POS tags
## checking the distribution of word count in title
fig = px.histogram(token_distribution_df, x="tags", y="count", title="Distribution POS Tags in Title")
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [21]:
## lets see how many words are alpha and how many are stop words
print(f"we have total {data['title_word_count'].sum()} words in the title. Out of which {is_alpha} are alpha and {is_stop} are stop words")

we have total 101176 words in the title. Out of which 100416 are alpha and 29908 are stop words


##### Notes
* So `maximum` tags are
    * `PROPN`- proper noun
    * `VERB` - verb
    * `ADP` - adposition
    * `NOUN` - noun
    * `PUNCT` - punctuation
    * `ADJ` - adjective
* Since these are news article titles, I think useful tags are, 
    * `PROPN`
    * `NOUN`
    * `VERB`
    * `ADJ` - Not sure about adjective yet. 
* We can remove rest of the words and still have a decent topic model. 
* We can also use the `is_stop` and `is_alpha` tags to remove the stopwords and non alpha tokens.
    * Lets update the helper functions accordingly. 

## Text Pre-processing

In [22]:
def preprocess_text(text):
     text = clean_html(text)
     text = lower_case(text)
     text = remove_line_breaks(text)
     text = remove_punctuation(text)
     text = remove_numbers(text)
     text = remove_extra_spaces(text)
     return text

data["cleaned_soft_title"] = data["soft_title"].apply(preprocess_text)

d:\workspace\Aletheia\env\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning:

The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.



### Tokenizing

In [23]:
data_words = list(sent_to_words(data['cleaned_soft_title']))

### Creating Bigram & Tigram Models

In [24]:
# Build the bigram and trigram models
bigram = Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[10]]])

['hall', 'of', 'famer', 'tim', 'hardaway', 'makes', 'distasteful', 'rape', 'comment', 'during', 'special', 'warriors', 'broadcast']


## Gensim LDA with BOW

### Lemmatization

In [25]:


# Form Bigrams
# data_words_bigrams = make_bigrams(data_words, bigram_mod)

# data_words_trigrams = make_trigrams(data_words, bigram_mod, trigram_mod)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words)

# Remove Stop Words
## we are removing stop words in the lemmatization function using spacy is_stop flag
# data_words_nostops = remove_stopwords(data_lemmatized)

print(data_lemmatized[:1])

[['biden', 'xi', 'meeting', 'attempt', 'china', 'taiwan']]


### Create Dictionary & Corpus

In [26]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [27]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]


[[('attempt', 1),
  ('biden', 1),
  ('china', 1),
  ('meeting', 1),
  ('taiwan', 1),
  ('xi', 1)]]

### Building the Topic Model

In [28]:
lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=50,
                     random_state=100,
                     update_every=1,
                     chunksize=200,
                     passes=200,
                    #  alpha='auto',
                     per_word_topics=True)


In [29]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

[(6,
  '0.181*"child" + 0.145*"crisis" + 0.093*"governor" + 0.084*"support" + '
  '0.076*"speech" + 0.056*"nick" + 0.036*"immigration" + 0.020*"brooks" + '
  '0.015*"garth" + 0.013*"cannon"'),
 (5,
  '0.125*"vaccine" + 0.110*"board" + 0.088*"response" + 0.054*"movie" + '
  '0.050*"halloween" + 0.040*"shot" + 0.019*"value" + 0.018*"years" + '
  '0.015*"disney" + 0.014*"cooper"'),
 (7,
  '0.181*"border" + 0.149*"official" + 0.108*"afghanistan" + 0.103*"policy" + '
  '0.091*"leader" + 0.089*"country" + 0.028*"attempt" + 0.027*"cheney" + '
  '0.020*"ally" + 0.017*"liz"'),
 (45,
  '0.192*"america" + 0.103*"violence" + 0.088*"tim" + 0.083*"tiktok" + '
  '0.043*"rape" + 0.036*"opponent" + 0.034*"hall" + 0.026*"marriage" + '
  '0.011*"guideline" + 0.000*"tune"'),
 (10,
  '0.115*"paul" + 0.079*"assault" + 0.077*"prison" + 0.071*"trial" + '
  '0.056*"company" + 0.052*"allegation" + 0.045*"harvey" + 0.039*"weinstein" + '
  '0.031*"fear" + 0.028*"press"'),
 (8,
  '0.379*"woman" + 0.064*"hour" + 0.

### Analyzing Model

In [30]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -38.80535460776146

Coherence Score:  0.5940622293009656


### Visualize Topics

In [31]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

d:\workspace\Aletheia\env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
49    -0.321038  0.302261       1        1  4.438262
19    -0.139124 -0.176666       2        1  3.208056
36    -0.132490 -0.158127       3        1  2.946877
22    -0.058854 -0.032532       4        1  2.830807
7     -0.108627 -0.096273       5        1  2.729969
43    -0.083639 -0.057986       6        1  2.676932
29    -0.084698 -0.059439       7        1  2.582850
3     -0.068256 -0.041139       8        1  2.504579
24    -0.022966 -0.008308       9        1  2.418067
47    -0.021196 -0.007235      10        1  2.402973
6     -0.032014 -0.013303      11        1  2.386969
15    -0.045015 -0.021923      12        1  2.361764
1     -0.032686 -0.013622      13        1  2.349286
16    -0.017195 -0.005146      14        1  2.341538
40    -0.006170 -0.000349      15        1  2.316065
33     0.014548  0.007939      16        1  2.266671
31    -0.047914 -0.023878      17        1  2.248289
48    -0.006610 -0.000536      18        1  2.233346
28    -0.005776 -0.000143      19        1  2.171673
34    -0.014094 -0.003770      20        1  2.148943
10     0.004623  0.004468      21        1  2.106579
42    -0.002104  0.001687      22        1  2.096253
37    -0.007779 -0.000942      23        1  2.087556
20     0.012002  0.006738      24        1  2.062429
46     0.004165  0.003980      25        1  2.037269
35     0.019979  0.009825      26        1  1.995590
0     -0.003385  0.001083      27        1  1.980898
32     0.003493  0.002767      28        1  1.974672
11     0.035980  0.014273      29        1  1.922578
38     0.031361  0.012976      30        1  1.875656
14     0.018694  0.009147      31        1  1.806788
2      0.023709  0.010919      32        1  1.804215
44     0.035322  0.013963      33        1  1.753633
45     0.051209  0.017719      34        1  1.684141
13     0.026548  0.011586      35        1  1.639154
26     0.043657  0.016122      36        1  1.618219
30     0.044531  0.016239      37        1  1.570648
5      0.061788  0.019745      38        1  1.561209
27     0.043990  0.016071      39        1  1.556119
21     0.037479  0.014423      40        1  1.505250
18     0.058712  0.019009      41        1  1.383698
17     0.062296  0.019721      42        1  1.374294
41     0.069042  0.020805      43        1  1.319172
4      0.073914  0.021282      44        1  1.317316
23     0.079571  0.021821      45        1  1.249804
8      0.068849  0.020603      46        1  1.248918
12     0.073743  0.021226      47        1  1.160754
9      0.075858  0.021414      48        1  1.148911
25     0.087526  0.021910      49        1  0.946704
39     0.099042  0.019595      50        1  0.647655, topic_info=         Term         Freq        Total Category  logprob  loglift
1       biden  1136.000000  1136.000000  Default  30.0000  30.0000
9         gop   502.000000   502.000000  Default  29.0000  29.0000
702    report   464.000000   464.000000  Default  28.0000  28.0000
10      house   444.000000   444.000000  Default  27.0000  27.0000
8    election   396.000000   396.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
24        law     0.041038   157.785521  Topic50  -8.9652  -3.2149
25       race     0.041038   392.407939  Topic50  -8.9652  -4.1260
27      state     0.041038   290.143737  Topic50  -8.9652  -3.8241
28   burglary     0.041038     8.845203  Topic50  -8.9652  -0.3336
29      crime     0.041038   264.394771  Topic50  -8.9652  -3.7311

[2418 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
62        9  0.992587     abortion
376       8  0.865018      academy
584      37  0.912749   accusation
598      47  0.959589      accuser
330       9  0.979487          act
...     ...       ...          ...
166      19  0.995559         year
788      38  0.871454        years
98       

## Gensim LDA with Bigram BOW

### Lemmatization

In [32]:
## lemmatization with bigrams
data_words_bigrams = make_bigrams(data_words, bigram_mod)

# data_words_trigrams = make_trigrams(data_words, bigram_mod, trigram_mod)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams)

# Remove Stop Words
## we are removing stop words in the lemmatization function using spacy is_stop flag
# data_words_nostops = remove_stopwords(data_lemmatized)

print(data_lemmatized[:1])

[['biden', 'xi', 'meeting', 'attempt', 'china', 'taiwan']]


### Create Dictionary & Corpus

In [33]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [34]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]


[[('attempt', 1),
  ('biden', 1),
  ('china', 1),
  ('meeting', 1),
  ('taiwan', 1),
  ('xi', 1)]]

### Building the Topic Model

In [35]:
lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=30,
                     random_state=100,
                     update_every=1,
                     chunksize=200,
                     passes=200,
                    #  alpha='auto',
                     per_word_topics=True)


In [36]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

[(22,
  '0.303*"gop" + 0.113*"voter" + 0.075*"mom" + 0.071*"deal" + 0.035*"power" + '
  '0.023*"speaker" + 0.019*"actor" + 0.017*"mccarthy" + 0.017*"bear" + '
  '0.017*"strategy"'),
 (4,
  '0.175*"state" + 0.165*"abortion" + 0.105*"iran" + 0.062*"issue" + '
  '0.046*"speech" + 0.034*"program" + 0.032*"protester" + 0.028*"nation" + '
  '0.023*"illinois" + 0.007*"extremism"'),
 (0,
  '0.259*"year" + 0.112*"america" + 0.099*"video" + 0.056*"baby" + '
  '0.033*"award" + 0.020*"album" + 0.010*"surprise" + 0.009*"ms" + '
  '0.007*"vmas" + 0.006*"name"'),
 (15,
  '0.232*"school" + 0.154*"family" + 0.113*"history" + 0.028*"vehicle" + '
  '0.021*"marine" + 0.013*"academy" + 0.011*"mtv" + 0.005*"lamborghini" + '
  '0.000*"theory" + 0.000*"teacher"'),
 (17,
  '0.124*"texas" + 0.113*"child" + 0.101*"bill" + 0.072*"gov" + 0.071*"threat" '
  '+ 0.068*"law" + 0.054*"support" + 0.050*"arizona" + 0.032*"dog" + '
  '0.011*"feud"'),
 (6,
  '0.176*"recipe" + 0.048*"decade" + 0.028*"fall" + 0.018*"pumpkin"

##### Notes
* So visually it seems we have a different topics when we use `bigrams`. 

### Analyzing Model

In [37]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -28.06815576710762

Coherence Score:  0.5763899628301856


### Visualize Topics

In [38]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

d:\workspace\Aletheia\env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
2     -0.258064 -0.240698       1        1  5.912750
23    -0.234753  0.287077       2        1  5.455741
21    -0.078747  0.038376       3        1  4.809168
17    -0.037005  0.019317       4        1  4.231169
20    -0.028390  0.016529       5        1  4.168703
19    -0.106555 -0.128385       6        1  3.994635
22    -0.008550  0.010054       7        1  3.978849
8     -0.034776  0.018437       8        1  3.968981
1      0.000196  0.007666       9        1  3.804418
14    -0.015949  0.012352      10        1  3.762284
4      0.011805  0.004493      11        1  3.591717
27     0.016620  0.003317      12        1  3.468274
7      0.008067  0.005520      13        1  3.448717
13     0.003212  0.006819      14        1  3.446855
11     0.022067  0.001987      15        1  3.185944
5      0.022875  0.001720      16        1  3.174656
29     0.034915 -0.000884      17        1  3.125945
0      0.041683 -0.002541      18        1  2.904757
12     0.040081 -0.002150      19        1  2.822833
24     0.050711 -0.004433      20        1  2.747036
16     0.048240 -0.003958      21        1  2.745527
15     0.048084 -0.003935      22        1  2.691312
10     0.045401 -0.003357      23        1  2.660644
26     0.052941 -0.004899      24        1  2.582031
3      0.054959 -0.005379      25        1  2.506581
28     0.056347 -0.005659      26        1  2.458337
18     0.060118 -0.006560      27        1  2.336036
25     0.058984 -0.006269      28        1  2.223961
9      0.061535 -0.006903      29        1  1.965074
6      0.063949 -0.007652      30        1  1.827065, topic_info=           Term         Freq        Total Category  logprob  loglift
1         biden  1015.000000  1015.000000  Default  30.0000  30.0000
182       trump   525.000000   525.000000  Default  29.0000  29.0000
9           gop   509.000000   509.000000  Default  28.0000  28.0000
635      report   449.000000   449.000000  Default  27.0000  27.0000
392      police   406.000000   406.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
17      arizona     0.071264    91.096027  Topic30  -9.2881  -3.1508
18   challenger     0.071264    33.684858  Topic30  -9.2881  -2.1560
19          gov     0.071264   130.428212  Topic30  -9.2881  -3.5097
20          law     0.071264   123.437191  Topic30  -9.2881  -3.4546
21         race     0.071264   286.026852  Topic30  -9.2881  -4.2950

[1265 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
55       11  0.991967       abortion
330      22  0.915664        academy
521       3  0.935144     accusation
535      12  0.950373        accuser
290      26  0.969132            act
174      24  0.973449       activist
594       7  0.958981          actor
92        1  0.965031             ad
136      20  0.909897           adam
407      29  0.884858        address
258       2  0.987269    afghanistan
536      21  0.783620      aftermath
658      13  0.965893          agent
287      17  0.949889            air
703      18  0.925098          album
369       1  0.878678           alex
353      17  0.973440     allegation
695       9  0.956125           ally
170      18  0.987018        america
299       5  0.982334      americans
267      29  0.966716         answer
402       1  0.793061            app
366       3  0.872239     appearance
17        4  0.976991        arizona
342      16  0.935780           army
76       24  0.829593            art
701       8  0.922008  assassination
240       6  0.974504        assault
453       4  0.900613        atlanta
0        12  0.965068        attempt
587      14  0.873337      attention
77       24  0.932169        auction
179       3  0.981291         austin
156      13  0.946292         author
352      17  0.947425      authority
80       18  0.969000     

## Gensim LDA with Trigram BOW

### Lemmatization

In [39]:
## lemmatization with trigrams
data_words_trigrams = make_trigrams(data_words, bigram_mod, trigram_mod)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams)

# Remove Stop Words
## we are removing stop words in the lemmatization function using spacy is_stop flag
# data_words_nostops = remove_stopwords(data_lemmatized)

print(data_lemmatized[:1])


[['biden', 'xi', 'meeting', 'attempt', 'china', 'taiwan']]


### Create Dictionary & Corpus

In [40]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [41]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]


[[('attempt', 1),
  ('biden', 1),
  ('china', 1),
  ('meeting', 1),
  ('taiwan', 1),
  ('xi', 1)]]

### Building the Topic Model

In [42]:
lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=30,
                     random_state=100,
                     update_every=1,
                     chunksize=200,
                     passes=200,
                    #  alpha='auto',
                     per_word_topics=True)


In [43]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

[(22,
  '0.166*"war" + 0.098*"home" + 0.078*"americans" + 0.050*"message" + '
  '0.037*"service" + 0.032*"nation" + 0.024*"thanksgiving" + 0.022*"minute" + '
  '0.018*"million" + 0.018*"legacy"'),
 (4,
  '0.273*"trump" + 0.100*"people" + 0.079*"twitter" + 0.074*"case" + '
  '0.048*"deal" + 0.047*"governor" + 0.047*"austin" + 0.025*"ban" + '
  '0.025*"assault" + 0.021*"weapon"'),
 (0,
  '0.190*"threat" + 0.042*"flight" + 0.026*"passenger" + 0.022*"atlanta" + '
  '0.020*"phone" + 0.012*"app" + 0.005*"iphone" + 0.000*"democracy" + '
  '0.000*"prescription" + 0.000*"columnist"'),
 (15,
  '0.072*"price" + 0.044*"park" + 0.043*"award" + 0.042*"hollywood" + '
  '0.041*"november" + 0.034*"truck" + 0.032*"ally" + 0.029*"diagnosis" + '
  '0.028*"toyota" + 0.025*"honor"'),
 (17,
  '0.123*"democrat" + 0.100*"victim" + 0.048*"district" + 0.045*"accuser" + '
  '0.033*"spending" + 0.032*"white" + 0.024*"th" + 0.022*"field" + 0.019*"mlb" '
  '+ 0.012*"republic"'),
 (6,
  '0.127*"veteran" + 0.101*"chri

### Analyzing Model

In [44]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -28.316860745843037

Coherence Score:  0.5671867602954693


### Visualize Topics

In [45]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

d:\workspace\Aletheia\env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
8     -0.244169 -0.277918       1        1  5.705021
29    -0.239436  0.286518       2        1  5.609682
28    -0.163987 -0.003206       3        1  5.283768
4     -0.085320 -0.001240       4        1  4.503991
13    -0.045580 -0.000812       5        1  4.363621
14    -0.035341 -0.000759       6        1  4.290194
11    -0.006071 -0.000473       7        1  3.776318
27    -0.007145 -0.000484       8        1  3.772756
1      0.003029 -0.000459       9        1  3.760547
24     0.007782 -0.000356      10        1  3.557253
12     0.013223 -0.000321      11        1  3.509030
21     0.014922 -0.000349      12        1  3.307667
18     0.018317 -0.000281      13        1  3.239999
5      0.010141 -0.000355      14        1  3.234913
2      0.031090 -0.000147      15        1  3.188324
22     0.026558 -0.000255      16        1  3.169635
25     0.035170 -0.000155      17        1  3.146060
19     0.037426 -0.000113      18        1  2.973249
20     0.037391 -0.000127      19        1  2.952102
7      0.025201 -0.000194      20        1  2.903179
9      0.047642 -0.000032      21        1  2.857385
3      0.048528 -0.000027      22        1  2.777820
23     0.047630  0.000003      23        1  2.691606
16     0.049037  0.000026      24        1  2.643245
17     0.059626  0.000172      25        1  2.323655
15     0.058367  0.000130      26        1  2.318449
26     0.059554  0.000185      27        1  2.270126
6      0.065040  0.000329      28        1  2.055114
10     0.064623  0.000302      29        1  2.004485
0      0.066755  0.000398      30        1  1.810807, topic_info=         Term         Freq        Total Category  logprob  loglift
1       biden  1024.000000  1024.000000  Default   30.000  30.0000
182     trump   510.000000   510.000000  Default   29.000  29.0000
9         gop   465.000000   465.000000  Default   28.000  28.0000
634    report   451.000000   451.000000  Default   27.000  27.0000
146      year   328.000000   328.000000  Default   26.000  26.0000
..        ...          ...          ...      ...      ...      ...
10      house     0.066369   266.371905  Topic30   -9.333  -4.2860
11        rep     0.066369   143.778564  Topic30   -9.333  -3.6694
12       seat     0.066369    37.266048  Topic30   -9.333  -2.3192
13     future     0.066369    32.803667  Topic30   -9.333  -2.1917
14   majority     0.066369    74.649826  Topic30   -9.333  -3.0139

[1263 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
55        7  0.993458       abortion
328      11  0.863949        academy
520      22  0.939502     accusation
534      25  0.960461        accuser
290      23  0.963527            act
174      12  0.977513       activist
593       4  0.944711          actor
92       24  0.967814             ad
136      28  0.900734           adam
405      15  0.878955        address
258       9  0.982900    afghanistan
535      21  0.857061      aftermath
657      17  0.954386          agent
287       3  0.953045            air
456      12  0.837058        airline
702      22  0.928308          album
367      17  0.869592           alex
351      21  0.967907     allegation
694      26  0.943916           ally
170      10  0.989551        america
299      16  0.982664      americans
267      19  0.942852         answer
400      30  0.852566            app
364      26  0.895977     appearance
17        5  0.979330        arizona
340      12  0.935284           army
76       29  0.877599            art
700       1  0.917783  assassination
240       4  0.954018        assault
452      30  0.915192        atlanta
0        23  0.950135        attempt
586      16  0.884191      attention
77       29  0.926218        auction
179       4  0.977062         austin
156      12  0.933435         author
350       3  0.913061      authority
80       26  

## Gensim LDA with Spacy noun-chunks n-Grams

##### Notes
* In this case we are trying to use `Spacy's` noun-chunks to create n-grams.
* We'll need to first create tokens from clean text. 
* We'll then need a function to replace the nouns in (noun chunks) with ngram word. 
* The `lemmatize` this, 
    * We'll need to test with and without POS to see if our filtering affects ngrams. 

### Creating Noun Chunks

In [46]:
## Lets start by reviewing noun chunks created by spacy
docs = [create_pos_tag(" ".join(x)) for x in data_words]

### Creating Noun Chunks Ngrams

In [47]:
n_grams = create_noun_chunk_ngrams(docs)
n_grams

[['biden',
  'says',
  'after',
  'xi',
  'meeting',
  'he',
  'doesn',
  'believe',
  'there',
  'is',
  'imminent_attempt',
  'from',
  'china',
  'to',
  'invade',
  'taiwan'],
 ['gop_rep_calvert',
  'wins',
  'election',
  'in',
  'competitive_california_house_seat'],
 ['pelosi',
  'not',
  'even',
  'thinking',
  'about',
  'political_future',
  'before',
  'house_majority',
  'called',
  'spox',
  'says'],
 ['arizona',
  'gov',
  'election',
  'katie_hobbs',
  'defeats',
  'gop_challenger',
  'kari_lake',
  'race',
  'may',
  'now',
  'go',
  'to',
  'recount',
  'under',
  'state_law'],
 ['idaho',
  'quadruple',
  'student',
  'homicide',
  'crime',
  'of',
  'passion',
  'burglary',
  'gone',
  'wrong',
  'among',
  'possible_motives',
  'mayor',
  'says'],
 ['jennifer_siebel_newsom',
  'identifies',
  'harvey_weinstein',
  'as',
  'alleged_rapist',
  'in',
  'emotional_sexual_assault_testimony'],
 ['commanders',
  'hand',
  'eagles',
  'first_loss',
  'of',
  'season',
  'in',

In [48]:
data_lemmatized = lemmatization_without_pos(n_grams)

In [49]:
data_lemmatized

[['biden',
  'say',
  'after',
  'xi',
  'meeting',
  'he',
  'doesn',
  'believe',
  'there',
  'be',
  'imminent_attempt',
  'from',
  'china',
  'to',
  'invade',
  'taiwan'],
 ['gop_rep_calvert',
  'win',
  'election',
  'in',
  'competitive_california_house_seat'],
 ['pelosi',
  'not',
  'even',
  'think',
  'about',
  'political_future',
  'before',
  'house_majority',
  'call',
  'spox',
  'say'],
 ['arizona',
  'gov',
  'election',
  'katie_hobbs',
  'defeat',
  'gop_challenger',
  'kari_lake',
  'race',
  'may',
  'now',
  'go',
  'to',
  'recount',
  'under',
  'state_law'],
 ['idaho',
  'quadruple',
  'student',
  'homicide',
  'crime',
  'of',
  'passion',
  'burglary',
  'go',
  'wrong',
  'among',
  'possible_motive',
  'mayor',
  'say'],
 ['jennifer_siebel_newsom',
  'identify',
  'harvey_weinstein',
  'as',
  'alleged_rapist',
  'in',
  'emotional_sexual_assault_testimony'],
 ['commander',
  'hand',
  'eagle',
  'first_loss',
  'of',
  'season',
  'in',
  'wild_nfc_east

##### Notes
* So since we are doing `lemmatization` without any `POS` some parts of speech words are present in our lemmatized data. 
* Words like `doesn't`, `believe`, `think` etc are present. I think we should update the function to ignore certain `POS` rather than just include all words. 
* Lets do a quick check on how these `POS` are distributed. 

In [50]:
n_gram_pos = [create_pos_tag(" ".join(x)) for x in data_lemmatized]
n_gram_pos_distribution, is_alpha, is_stop = create_pos_tags_distribution(n_gram_pos)

In [51]:
token_distribution_df = pd.DataFrame.from_dict(n_gram_pos_distribution, orient='index', columns=['count']).reset_index().rename(columns={"index": "tags"})
## lets create a distribution of POS tags
## checking the distribution of word count in title
fig = px.histogram(token_distribution_df, x="tags", y="count", title="Distribution POS Tags in Title")
fig.show()

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

d:\workspace\Aletheia\env\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



##### Notes
* Lets see how our `ngrams` are tagged

In [52]:
for token in n_gram_pos:
    for token in token:
        ## Only print the noun chunks
        if("_" in token.text):            
            print(token.text, token.pos_, token.dep_)

imminent_attempt NOUN attr
gop_rep_calvert PROPN nsubj
competitive_california_house_seat PROPN pobj
political_future NOUN pobj
house_majority NOUN dep
katie_hobbs PROPN appos
gop_challenger NOUN dobj
kari_lake PROPN conj
state_law NOUN pobj
possible_motive ADJ amod
jennifer_siebel_newsom PROPN nsubj
harvey_weinstein PROPN dobj
alleged_rapist NOUN pobj
emotional_sexual_assault_testimony NOUN pobj
first_loss NOUN appos
wild_nfc_east_upset NOUN pobj
arizona_gop_rep_schweikert PROPN dep
democratic_challenger PROPN dobj
house_majority PROPN pobj
marjorie_taylor_greene PROPN dep
gop_colleague PROPN pobj
speaker_bad_strategy PROPN pobj
student_loan_buyout_abortion NOUN nsubj
distasteful_rape_comment ADJ dobj
special_warriors_broadcast PROPN pobj
oregons_th_congressional_district ADJ amod
democrat_andrea_salinas PROPN pobj
andy_warhol_art NOUN pobj
kevin_costner PROPN nsubj
yellowstone_national_park PROPN dobj
patriot_awards PROPN npadvmod
honor_heroism NOUN dobj
fox_nation PROPN pobj
this_day

##### Notes
* Looks like most of the noun chunks are tagged as `NOUN`, `PROPN` or `ADJ`, but lets confirm it using visualization. 


In [53]:
noun_chunks_tags = {}
for token in n_gram_pos:
    for token in token:
        ## Only print the noun chunks
        if("_" in token.text):
            ## increment the count of the noun chunk
            noun_chunks_tags[token.pos_] = noun_chunks_tags.get(token.pos_, 0) + 1

noun_chunks_tags

{'NOUN': 8112,
 'PROPN': 8452,
 'ADJ': 714,
 'X': 851,
 'VERB': 74,
 'NUM': 30,
 'ADV': 10,
 'PRON': 56,
 'INTJ': 4,
 'SYM': 4,
 'ADP': 1,
 'DET': 1}

##### Notes
* So as assumed most of them are `NOUN` or `PROPN` but just to make sure we don't loose any chunks, lets modify our lemmatization script to handle this case. 

In [54]:
data_lemmatized = lemmatization_noun_chunks(n_grams)

In [55]:
data_lemmatized

[['biden', 'xi', 'meeting', 'imminent_attempt', 'china', 'taiwan'],
 ['gop_rep_calvert', 'election', 'competitive_california_house_seat'],
 ['pelosi', 'political_future', 'house_majority', 'spox'],
 ['arizona',
  'gov',
  'election',
  'katie_hobbs',
  'gop_challenger',
  'kari_lake',
  'race',
  'recount',
  'state_law'],
 ['idaho',
  'student',
  'homicide',
  'crime',
  'passion',
  'burglary',
  'possible_motive',
  'mayor'],
 ['jennifer_siebel_newsom',
  'harvey_weinstein',
  'alleged_rapist',
  'emotional_sexual_assault_testimony'],
 ['commander', 'eagle', 'first_loss', 'season', 'wild_nfc_east_upset'],
 ['arizona_gop_rep_schweikert',
  'democratic_challenger',
  'republicans',
  'house_majority'],
 ['marjorie_taylor_greene',
  'gop_colleague',
  'mccarthy',
  'speaker_bad_strategy'],
 ['student_loan_buyout_abortion', 'midterm', 'democrats', 'biden', 'critic'],
 ['hall',
  'famer',
  'tim',
  'hardaway',
  'distasteful_rape_comment',
  'special_warriors_broadcast'],
 ['oregons_th

##### Notes
* So the lemmatized data looks promising lets train the model and see. 

### Create Dictionary & Corpus

In [58]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [59]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]


[[('biden', 1),
  ('china', 1),
  ('imminent_attempt', 1),
  ('meeting', 1),
  ('taiwan', 1),
  ('xi', 1)]]

### Building the Topic Model

In [60]:
lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=30,
                     random_state=100,
                     update_every=1,
                     chunksize=200,
                     passes=200,
                    #  alpha='auto',
                     per_word_topics=True)


In [61]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

[(18,
  '0.079*"america" + 0.020*"fox_nation" + 0.019*"baby" + 0.007*"flight" + '
  '0.006*"kevin_costner" + 0.000*"massive_shift" + 0.000*"nightmare_scenario" '
  '+ 0.000*"mccaul" + 0.000*"nato_majority" + 0.000*"red_line"'),
 (17,
  '0.156*"year" + 0.045*"threat" + 0.027*"life" + 0.010*"kathie_lee_gifford" + '
  '0.007*"cher" + 0.004*"atlanta" + 0.000*"democracy" + 0.000*"massive_shift" '
  '+ 0.000*"possible_threat" + 0.000*"nato_majority"'),
 (20,
  '0.009*"joe_biden" + 0.008*"loss" + 0.006*"chief" + 0.005*"the_nation" + '
  '0.000*"mccaul" + 0.000*"nato_majority" + 0.000*"massive_shift" + '
  '0.000*"quantum_computing_answer" + 0.000*"nuclear_weapons" + '
  '0.000*"possible_threat"'),
 (23,
  '0.027*"veteran" + 0.016*"christmas" + 0.015*"thanksgiving" + '
  '0.014*"tribute" + 0.009*"jeff" + 0.008*"team" + 0.003*"tom_brady" + '
  '0.002*"saturday" + 0.000*"massive_shift" + 0.000*"mccaul"'),
 (10,
  '0.085*"iran" + 0.000*"massive_shift" + 0.000*"red_line" + '
  '0.000*"nuclear_weap

### Analyzing Model

In [62]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -48.68804192741965

Coherence Score:  0.5222654255319291


##### Notes
* So our model didn't perform well, in fact regular model BOW did better than noun chunks. 

### Visualize Topics

In [63]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

d:\workspace\Aletheia\env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13     0.146446 -0.124518       1        1  5.383394
22    -0.147752 -0.123232       2        1  5.279413
12     0.000172  0.033807       3        1  3.997223
7      0.000138  0.026461       4        1  3.959367
17     0.000145  0.028044       5        1  3.818324
28     0.000095  0.017702       6        1  3.599469
9      0.000097  0.018143       7        1  3.562110
11     0.000105  0.019678       8        1  3.539652
1      0.000078  0.014193       9        1  3.475027
8      0.000071  0.012979      10        1  3.420717
4      0.000088  0.016261      11        1  3.398471
14     0.000090  0.016677      12        1  3.372617
15     0.000051  0.009251      13        1  3.322350
29     0.000060  0.011018      14        1  3.238948
18     0.000039  0.007153      15        1  3.200399
26     0.000052  0.009520      16        1  3.173025
6      0.000044  0.008080      17        1  3.145606
21     0.000027  0.005040      18        1  3.135344
2      0.000022  0.004076      19        1  3.014306
23     0.000015  0.002799      20        1  2.999242
24     0.000011  0.002080      21        1  2.973060
0      0.000010  0.001826      22        1  2.970302
5      0.000005  0.001183      23        1  2.941265
27     0.000005  0.000999      24        1  2.915346
10     0.000010  0.002002      25        1  2.810244
25    -0.000007 -0.000969      26        1  2.782167
20    -0.000025 -0.004309      27        1  2.697947
19    -0.000019 -0.003147      28        1  2.697408
3     -0.000032 -0.005407      29        1  2.644502
16    -0.000043 -0.007390      30        1  2.532754, topic_info=            Term        Freq       Total Category  logprob  loglift
0          biden  566.000000  566.000000  Default  30.0000  30.0000
552        trump  282.000000  282.000000  Default  29.0000  29.0000
85       ukraine  183.000000  183.000000  Default  28.0000  28.0000
113         year  185.000000  185.000000  Default  27.0000  27.0000
46     democrats  149.000000  149.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
16     kari_lake    0.044116   10.784575  Topic30  -9.7827  -1.8232
17   katie_hobbs    0.044116    3.921392  Topic30  -9.7827  -0.8115
18          race    0.044116   48.846987  Topic30  -9.7827  -3.3338
22         crime    0.044116   61.232429  Topic30  -9.7827  -3.5597
28       student    0.044116   39.296899  Topic30  -9.7827  -3.1162

[1242 rows x 6 columns], token_table=      Topic      Freq                   Term
term                                        
432      29  0.820751             accusation
131      15  0.982652                america
213       2  0.916608                 answer
13        8  0.956293                arizona
553      11  0.776839                   army
376       5  0.843243                atlanta
58        8  0.927979                auction
530      13  0.911639                 author
287      19  0.518465              authority
117      15  0.908312                   baby
695      13  0.693719  baby_formula_shortage
259      11  0.954207                 battle
0         1  0.998337                  biden
483      21  0.929532                 bidens
660      22  0.893289            bill_murray
693       7  0.934424                  birth
669      17  0.858994                brother
410      28  0.928670             california
381      13  0.937720                   call
502       9  0.918733                    car
708      22  0.728086                 chance
390       5  0.843451                   cher
440      27  0.754134                  chief
1        17  0.986006                  china
515      20  0.912598              christmas
33       21  0.788694              commander
305      26  0.530893  commanders_dan_snyder
452      21  0.921201                  court
22       14  0.979873                  crime
45        4 

## Sklearn LDA with Count Vectorization

##### Notes
* For Count Vecotorization we'l use SKLearn's LDA algorithm. The algorithm is same as Gensim, but the interface is different and it allows us to use CountVectorizations. 

### Lemmatization

In [65]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words)


print(data_lemmatized[:1])

[['biden', 'xi', 'meeting', 'attempt', 'china', 'taiwan']]


### Count Vectorizer

In [66]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform([" ".join(lem_word) for lem_word in data_lemmatized])

In [67]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")


Sparsicity:  0.41396974461688885 %


### Building Topic Model

In [68]:
# Build LDA Model 35
lda_model = LatentDirichletAllocation(n_components=33,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=33, n_jobs=-1,
                          random_state=100)


### Analyzing Model

In [69]:
analyze_sklearn_lda_model(lda_model, data_vectorized)
# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -236788.08924215176
Perplexity:  1282.4523594796467
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 33,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [70]:
dominant_topics = print_sklearn_dominant_topics(lda_model, data_vectorized)
dominant_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,Topic25,Topic26,Topic27,Topic28,Topic29,Topic30,Topic31,Topic32,dominant_topic
Doc0,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.840000,0.010000,31
Doc1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.860000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25
Doc2,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.210000,0.010000,0.010000,0.010000,0.010000,0.010000,0.210000,0.010000,0.010000,0.410000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,25
Doc3,0.000000,0.000000,0.000000,0.930000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3
Doc4,0.010000,0.010000,0.010000,0.010000,0.010000,0.210000,0.010000,0.010000,0.010000,0.610000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,9
Doc5,0.010000,0.010000,0.010000,0.010000,0.010000,0.510000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.340000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,5
Doc6,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.210000,0.010000,0.010000,0.010000,0.010000,0.610000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,24
Doc7,0.000000,0.130000,0.000000,0.340000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25
Doc8,0.010000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.510000,0.010000,0.010000,0.010000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,16
Doc9,0.000000,0.000000,0.000000,0.000000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.130000,0.000000,5


In [71]:
topic_distribution = print_sklearn_topic_distribution(lda_model, data_vectorized)
topic_distribution

,Topic,Num Documents
0,3,688
1,6,601
2,5,573
3,0,550
4,10,510
5,1,486
6,16,442
7,9,364
8,2,343
9,8,327


### Visualizing Topics

In [74]:
panel = visualize_sklearn_lda_model(lda_model, data_vectorized, vectorizer)
panel

d:\workspace\Aletheia\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.

d:\workspace\Aletheia\env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.

d:\workspace\Aletheia\env\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

d:\workspace\Aletheia\env\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



PreparedData(topic_coordinates=               x          y  topics  cluster      Freq
topic                                                 
16    -34.027336  18.225710       1        1  6.054213
3     -22.827232   1.109005       2        1  5.865510
10      0.780303  17.086075       3        1  5.698954
25    -20.786575 -31.716217       4        1  5.642586
31     46.194942  31.609703       5        1  5.314847
5      15.202925 -42.588188       6        1  4.833790
6      46.114040 -21.349285       7        1  4.761404
22      9.766649  46.719769       8        1  4.668092
24    -15.471023 -13.383763       9        1  4.458055
29     26.981581 -17.243277      10        1  4.365406
20     28.288507  25.070833      11        1  4.185709
9     -40.931793   0.203981      12        1  4.017324
8      55.224182  -2.278610      13        1  3.485081
14     45.096191  13.268657      14        1  3.416285
0      -9.630401  47.950550      15        1  3.247423
17     -2.714104 -23.255640      16        1  3.120893
1      12.214280  28.931179      17        1  3.061853
19    -23.670996  33.861713      18        1  3.005465
13     37.283749  -4.353387      19        1  2.599245
2      31.529617 -33.834541      20        1  2.461047
23     -3.128274 -40.058384      21        1  2.400034
15     27.941891  42.498642      22        1  2.242873
30    -32.105846 -16.842770      23        1  1.749479
11     -4.595796  31.043821      24        1  1.636367
12    -15.648289  17.298254      25        1  1.358484
27     12.817910 -24.638971      26        1  1.084122
4      21.279423  -1.827757      27        1  0.949183
26     28.697817   9.838983      28        1  0.914904
28     -7.429880   4.589966      29        1  0.852098
32     14.817351  13.150508      30        1  0.676820
21     11.163427 -10.196991      31        1  0.650156
18     -1.680116  -7.664780      32        1  0.646897
7       6.963400   3.013175      33        1  0.575402, topic_info=         Term        Freq       Total Category  logprob  loglift
90      biden  593.000000  593.000000  Default  30.0000  30.0000
762    report  266.000000  266.000000  Default  29.0000  29.0000
422     house  270.000000  270.000000  Default  28.0000  28.0000
821    senate  208.000000  208.000000  Default  27.0000  27.0000
947     trump  250.000000  250.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
139       cdc    0.124977   25.961907  Topic33  -7.3287  -0.1784
868     steve    0.124977   18.643561  Topic33  -7.3287   0.1527
902    taylor    0.124977   76.828970  Topic33  -7.3287  -1.2633
144  ceremony    0.124977   15.756377  Topic33  -7.3287   0.3210
591      mile    0.124977   12.668613  Topic33  -7.3287   0.5391

[1286 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0        21  0.948299     abbott
2         6  0.994295   abortion
3         4  0.975496     abrams
4         6  0.928627      abuse
5        11  0.919734    academy
...     ...       ...        ...
1024      1  0.987320       york
1025     17  0.894100   youngkin
1026      2  0.929607     zeldin
1027     12  0.926182  zelenskyy
1028     13  0.939574     zucker

[853 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 4, 11, 26, 32, 6, 7, 23, 25, 30, 21, 10, 9, 15, 1, 18, 2, 20, 14, 3, 24, 16, 31, 12, 13, 28, 5, 27, 29, 33, 22, 19, 8])

In [81]:
## vectorizer, lda_model, n_words=20
df_topic_keywords = show_sklearn_topics(vectorizer, lda_model)
formatted_topics = format_sklearn_topics(df_topic_keywords)
formatted_topics

d:\workspace\Aletheia\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,senate,news,fox,media,reporter,christmas,power,control,service,nevada,hero,robert,risk,ceo,bee,scott,showdown,battleground,thanksgiving,stroke
Topic 1,death,republicans,iran,recipe,nyc,week,fbi,party,protester,head,regime,leadership,singer,boss,idea,run,chicken,cup,mcconnell,site
Topic 2,woman,world,baby,taliban,food,mass,shortage,olivia,hour,formula,birth,wilde,film,debt,shia,flight,kabul,secret,labeouf,united
Topic 3,race,election,gop,candidate,gov,medium,law,security,state,arizona,brian,force,msnbc,result,stelter,meghan,reelection,senator,view,hochul
Topic 4,son,test,drive,rock,toyota,jeep,cancer,ford,mercedesbenz,covid,chinas,pickup,elizabeth,deal,father,policy,sister,meghan,anniversary,dad
Topic 5,abortion,covid,student,times,charge,paul,assault,member,trial,vaccine,cuomo,sex,act,allegation,harvey,loan,rape,handout,weinstein,accuser
Topic 6,ukraine,russia,war,state,florida,threat,putin,issue,month,desantis,invasion,south,chief,drone,journalist,anne,shark,defense,visit,agent
Topic 7,fauci,right,kyiv,datum,maloney,impeachment,truck,nbcs,university,supporter,mtv,regime,los,run,trump,end,supreme,nicole,pentagon,dinner
Topic 8,cnn,man,americans,chris,scandal,voting,drug,sheriff,friend,dad,use,sanders,missile,supporter,prosecutor,laptop,jet,island,zucker,alaska
Topic 9,crime,president,project,parent,murder,lincoln,mayor,suspect,study,college,photo,tim,number,ryan,netflix,carolina,cofounder,north,hall,harassment


##### Notes
* Somehow I couldn't find out `coherence` for but from visualization it seems that the model has created topics which are easy to interpret and not overlapping
* I still see some topics that don't make sense but, we can tweak this further by creating n-grams, including more POS and skipping lemmatization. 

### Write to Pickle File

In [88]:
# create an iterator object with write permission - model.pkl
with open('../pickles/sklean_count_vectorization_v1', 'wb') as files:
    pickle.dump(lda_model, files)

## Sklearn LDA with n-grams Count Vectorization

### Lemmatization with Bigrams

In [89]:
## lemmatization with bigrams
data_words_bigrams = make_bigrams(data_words, bigram_mod)

# data_words_trigrams = make_trigrams(data_words, bigram_mod, trigram_mod)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
## performaing lemmatization with noun chunks to preserve the ngram words. 
data_lemmatized = lemmatization_noun_chunks(data_words_bigrams)

# Remove Stop Words
## we are removing stop words in the lemmatization function using spacy is_stop flag
# data_words_nostops = remove_stopwords(data_lemmatized)

print(data_lemmatized[:1])

[['biden', 'xi', 'meeting', 'attempt', 'china', 'taiwan']]


In [90]:
data_lemmatized

[['biden', 'xi', 'meeting', 'attempt', 'china', 'taiwan'],
 ['gop', 'rep', 'calvert', 'election', 'california', 'house', 'seat'],
 ['pelosi', 'future', 'house', 'majority', 'spox'],
 ['arizona',
  'gov',
  'election',
  'katie_hobbs',
  'gop',
  'challenger',
  'kari_lake',
  'race',
  'recount',
  'state',
  'law'],
 ['idaho',
  'student',
  'homicide',
  'crime',
  'passion',
  'burglary',
  'motive',
  'mayor'],
 ['jennifer',
  'siebel',
  'newsom',
  'harvey_weinstein',
  'rapist',
  'sexual_assault',
  'testimony'],
 ['commander', 'eagles', 'loss', 'season', 'wild', 'nfc', 'east', 'upset'],
 ['arizona',
  'gop',
  'rep',
  'schweikert',
  'challenger',
  'republicans',
  'house',
  'majority'],
 ['marjorie',
  'taylor',
  'greene',
  'gop',
  'colleague',
  'mccarthy',
  'speaker',
  'bad',
  'strategy'],
 ['student_loan',
  'buyout',
  'abortion',
  'midterm',
  'democrats',
  'biden',
  'critic'],
 ['hall',
  'famer',
  'tim',
  'hardaway',
  'rape',
  'comment',
  'special',
  

### Count Vectorizer

In [91]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform([" ".join(lem_word) for lem_word in data_lemmatized])

In [92]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")


Sparsicity:  0.4066095871207507 %


### Building Topic Model

In [93]:
# Build LDA Model 35
lda_model = LatentDirichletAllocation(n_components=33,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=33, n_jobs=-1,
                          random_state=100)


### Analyzing Model

In [94]:
analyze_sklearn_lda_model(lda_model, data_vectorized)

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -240794.79689144716
Perplexity:  1228.5419817944949
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 33,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


### Visualizing Topics

In [95]:
dominant_topics = print_sklearn_dominant_topics(lda_model, data_vectorized)
dominant_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,Topic25,Topic26,Topic27,Topic28,Topic29,Topic30,Topic31,Topic32,dominant_topic
Doc0,0.010000,0.010000,0.010000,0.010000,0.010000,0.840000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,5
Doc1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.860000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20
Doc2,0.010000,0.010000,0.810000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,2
Doc3,0.930000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
Doc4,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.210000,0.010000,0.010000,0.610000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,21
Doc5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.580000,0.000000,0.000000,0.000000,0.000000,0.290000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9
Doc6,0.010000,0.210000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.610000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,26
Doc7,0.260000,0.000000,0.140000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.130000,0.360000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20
Doc8,0.010000,0.010000,0.510000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.170000,0.010000,2
Doc9,0.630000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0


In [96]:
topic_distribution = print_sklearn_topic_distribution(lda_model, data_vectorized)
topic_distribution

,Topic,Num Documents
0,0,1010
1,5,700
2,2,638
3,7,518
4,1,413
5,4,383
6,26,336
7,18,320
8,14,312
9,11,298


In [97]:
panel = visualize_sklearn_lda_model(lda_model, data_vectorized, vectorizer)
panel

d:\workspace\Aletheia\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.

d:\workspace\Aletheia\env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.

d:\workspace\Aletheia\env\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

d:\workspace\Aletheia\env\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
0       24.041759  -96.092506       1        1  7.021072
5     -168.012726  -67.394791       2        1  5.740797
26      30.240131  151.618530       3        1  5.712465
7     -118.418755  -29.350657       4        1  5.027230
2      140.090057   81.206413       5        1  4.996198
31     -80.189262 -160.233276       6        1  4.763897
25    -111.270615 -103.225769       7        1  4.432422
18     124.800278 -119.918205       8        1  4.366583
14     144.542831  -52.997993       9        1  4.200853
20      76.460754  -75.072716      10        1  3.994120
21      82.657227   46.759129      11        1  3.793243
11     -41.174534  158.990906      12        1  3.671716
27    -110.850380  137.613998      13        1  3.491309
1      142.467651   14.056152      14        1  3.344577
4     -171.637543   12.096598      15        1  3.309953
19      57.549473 -149.795746      16        1  3.298459
13    -146.827438   76.473763      17        1  3.099094
17      90.544518  127.381485      18        1  3.072071
23     -15.837448  100.254166      19        1  2.935458
30     -36.111797 -103.997482      20        1  2.776081
8       -9.357663 -160.301300      21        1  2.441316
22      85.611725  -15.800694      22        1  2.300203
9      -74.979904   86.465088      23        1  1.992706
16    -103.227638   28.687717      24        1  1.950270
15      44.120354   81.801857      25        1  1.762141
12     -69.709618  -59.132065      26        1  1.182243
32     -47.182243   40.902138      27        1  0.949929
28      30.652615  -38.341503      28        1  0.861789
24     -10.572631   -0.599380      29        1  0.720138
3        1.871786   47.241611      30        1  0.719360
10      36.990501   12.280583      31        1  0.719253
6      -58.116253   -8.681837      32        1  0.698349
29     -18.237486  -47.333969      33        1  0.654705, topic_info=         Term        Freq       Total Category  logprob  loglift
791    report  255.000000  255.000000  Default  30.0000  30.0000
90      biden  600.000000  600.000000  Default  29.0000  29.0000
62     attack  197.000000  197.000000  Default  28.0000  28.0000
985     trump  260.000000  260.000000  Default  27.0000  27.0000
383       gop  278.000000  278.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
32       ally    0.079271   15.876958  Topic33  -7.9358  -0.2710
1037     wave    0.079271   20.389879  Topic33  -7.9358  -0.5212
484       joe    0.079271   46.352927  Topic33  -7.9358  -1.3424
688   opinion    0.079271   10.411559  Topic33  -7.9358   0.1509
326      fame    0.079271   13.160189  Topic33  -7.9358  -0.0833

[1292 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         4  0.927843     abbott
1        21  0.929995        abc
2         1  0.344696   abortion
2         9  0.648495   abortion
3         3  0.967739     abrams
...     ...       ...        ...
1067     13  0.989214       york
1068      9  0.915619   youngkin
1069     11  0.935129     zeldin
1070     11  0.966405  zelenskyy
1071     21  0.928991     zucker

[895 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 27, 8, 3, 32, 26, 19, 15, 21, 22, 12, 28, 2, 5, 20, 14, 18, 24, 31, 9, 23, 10, 17, 16, 13, 33, 29, 25, 4, 11, 7, 30])

In [99]:
## vectorizer, lda_model, n_words=20
df_topic_keywords = show_sklearn_topics(vectorizer, lda_model)
formatted_topics = format_sklearn_topics(df_topic_keywords)
formatted_topics

d:\workspace\Aletheia\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,biden,midterm,poll,gov,medium,election,law,race,gun,pennsylvania,state,abortion,democrats,theory,county,right,arizona,critical,plan,judge
Topic 1,claim,campaign,week,georgia,msnbc,walker,voting,governor,ford,senate,committee,election,herschel,warnock,yellowstone,leadership,commander,mcconnell,boss,actress
Topic 2,voter,time,world,pelosi,america,debate,protest,majority,activist,nation,music,message,drug,lawsuit,economy,investigation,ryan,house,los,angeles
Topic 3,stelter,kavanaugh,europe,israel,ted,kabul,portland,zeldin,united,high,founder,controversy,joke,agent,sheriff,tour,cartel,face,year,hollywood
Topic 4,day,history,security,force,michigan,interview,cnns,west,military,national,justice,elvis,air,americas,kanye,presley,chief,scotus,tapper,backlash
Topic 5,border,biden,china,official,migrant,coronavirus,city,video,policy,deal,taiwan,admin,office,union,illegal,tweet,southern,source,number,visit
Topic 6,trumps,maralago,master,step,special,concern,illinois,high,georgia,value,army,mike,cheating,prison,cruz,list,bidens,york,lie,intelligence
Topic 7,texas,news,fox,biden,media,hunter,car,dems,george,price,host,power,gas,tucker,drone,victory,view,girl,business,journalist
Topic 8,cnn,expert,book,study,scandal,queen,prince,king,charles,harry,criticism,heart,zucker,nbc,doctor,author,elizabeth,yearold,abc,shows
Topic 9,star,test,wife,trial,harvey,drive,rape,weinstein,celebrity,years,performance,divorce,newsom,stroke,jennifer,toyota,later,place,jeep,value


### Write to Pickle File

In [126]:
# create an iterator object with write permission - model.pkl
with open('../pickles/sklearn_count_vectorization_bi_grams_v1', 'wb') as files:
    pickle.dump(lda_model, files)

## SKLearn LDA with noun-chunks n-grams

### Creating Noun Chunks

In [101]:
## Lets start by reviewing noun chunks created by spacy
docs = [create_pos_tag(" ".join(x)) for x in data_words]

### Creating Noun Chunks N-grams

In [102]:
n_grams = create_noun_chunk_ngrams(docs)

In [103]:
## lemmatization with bigrams
# data_words_bigrams = make_bigrams(data_words, bigram_mod)

# data_words_trigrams = make_trigrams(data_words, bigram_mod, trigram_mod)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
## performaing lemmatization with noun chunks to preserve the ngram words. 
data_lemmatized = lemmatization_noun_chunks(n_grams)

# Remove Stop Words
## we are removing stop words in the lemmatization function using spacy is_stop flag
# data_words_nostops = remove_stopwords(data_lemmatized)

print(data_lemmatized[:1])

[['biden', 'xi', 'meeting', 'imminent_attempt', 'china', 'taiwan']]


In [104]:
data_lemmatized

[['biden', 'xi', 'meeting', 'imminent_attempt', 'china', 'taiwan'],
 ['gop_rep_calvert', 'election', 'competitive_california_house_seat'],
 ['pelosi', 'political_future', 'house_majority', 'spox'],
 ['arizona',
  'gov',
  'election',
  'katie_hobbs',
  'gop_challenger',
  'kari_lake',
  'race',
  'recount',
  'state_law'],
 ['idaho',
  'student',
  'homicide',
  'crime',
  'passion',
  'burglary',
  'possible_motive',
  'mayor'],
 ['jennifer_siebel_newsom',
  'harvey_weinstein',
  'alleged_rapist',
  'emotional_sexual_assault_testimony'],
 ['commander', 'eagle', 'first_loss', 'season', 'wild_nfc_east_upset'],
 ['arizona_gop_rep_schweikert',
  'democratic_challenger',
  'republicans',
  'house_majority'],
 ['marjorie_taylor_greene',
  'gop_colleague',
  'mccarthy',
  'speaker_bad_strategy'],
 ['student_loan_buyout_abortion', 'midterm', 'democrats', 'biden', 'critic'],
 ['hall',
  'famer',
  'tim',
  'hardaway',
  'distasteful_rape_comment',
  'special_warriors_broadcast'],
 ['oregons_th

### Count Vectorizer

In [105]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform([" ".join(lem_word) for lem_word in data_lemmatized])

In [106]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")


Sparsicity:  0.3931762420608856 %


### Building Topic Model

In [107]:
# Build LDA Model 35
lda_model = LatentDirichletAllocation(n_components=33,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=33, n_jobs=-1,
                          random_state=100)


### Analyzing Model

In [108]:
analyze_sklearn_lda_model(lda_model, data_vectorized)

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -261313.12504390915
Perplexity:  1492.5352854842884
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 33,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


### Visualizing Topics

In [109]:
dominant_topics = print_sklearn_dominant_topics(lda_model, data_vectorized)
dominant_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,Topic25,Topic26,Topic27,Topic28,Topic29,Topic30,Topic31,Topic32,dominant_topic
Doc0,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.190000,0.320000,0.010000,0.010000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,19
Doc1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.220000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23
Doc2,0.010000,0.010000,0.150000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.510000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.190000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,9
Doc3,0.000000,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.770000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20
Doc4,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.670000,32
Doc5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.880000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13
Doc6,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.170000,0.510000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.170000,24
Doc7,0.110000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.580000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19
Doc8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.150000,0.720000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16
Doc9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.130000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20


In [110]:
topic_distribution = print_sklearn_topic_distribution(lda_model, data_vectorized)
topic_distribution

,Topic,Num Documents
0,2,746
1,0,644
2,15,607
3,20,605
4,4,531
5,5,508
6,23,446
7,9,442
8,12,272
9,3,267


In [111]:
panel = visualize_sklearn_lda_model(lda_model, data_vectorized, vectorizer)
panel

d:\workspace\Aletheia\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.

d:\workspace\Aletheia\env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.

d:\workspace\Aletheia\env\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

d:\workspace\Aletheia\env\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
20     129.985184   67.332825       1        1  7.951151
15      72.179077  -90.480339       2        1  6.646269
2      -18.381920  118.224792       3        1  5.995438
23      97.562584  124.561905       4        1  5.858146
24    -146.654861   75.350273       5        1  4.991397
5       65.976578 -150.726639       6        1  4.576250
19     -10.408031 -164.175079       7        1  4.274186
9     -166.506683    9.216008       8        1  4.153665
32    -108.591011 -130.021118       9        1  4.144756
25     150.913162   -4.668108      10        1  4.123910
4      137.304047  -80.304512      11        1  4.077267
0       10.544792  173.834702      12        1  3.749512
18      45.834019  118.749825      13        1  3.575096
16      62.353477   61.808704      14        1  3.424484
27    -149.796143  -66.098312      15        1  3.172955
12      89.848366  -36.618107      16        1  2.886779
26     -85.166565  -67.248901      17        1  2.880113
17      -6.502284  -19.444307      18        1  2.729090
11     -72.034302  153.240692      19        1  2.465878
3       12.391507 -103.360954      20        1  2.462579
22     -91.236366   35.970776      21        1  2.429543
14     -47.621502 -113.814079      22        1  2.406270
13     -88.876923   95.506569      23        1  2.187818
30    -111.193329  -14.444450      24        1  2.057662
8       90.230812   18.015636      25        1  1.297498
10     -27.689854  -61.359314      26        1  0.769250
29       8.148563   69.901314      27        1  0.767003
6       29.394857  -51.628036      28        1  0.732824
31     -42.728455   65.200455      29        1  0.715248
28     -57.904228  -17.368713      30        1  0.657286
21     -32.838566   20.978540      31        1  0.632169
1       13.622807   25.591089      32        1  0.606318
7       43.449928   -5.364204      33        1  0.602190, topic_info=           Term        Freq       Total Category  logprob  loglift
94        biden  609.000000  609.000000  Default   30.000  30.0000
727         new  401.000000  401.000000  Default   29.000  29.0000
880      report  237.000000  237.000000  Default   28.000  28.0000
190         cnn  157.000000  157.000000  Default   27.000  27.0000
1075      trump  259.000000  259.000000  Default   26.000  26.0000
...         ...         ...         ...      ...      ...      ...
108        book    0.085186   51.882203  Topic33   -7.835  -1.2995
90         best    0.085186   25.586677  Topic33   -7.835  -0.5926
814   potential    0.085186   21.840595  Topic33   -7.835  -0.4343
50    antitrump    0.085186   17.085957  Topic33   -7.835  -0.1888
853        race    0.085186  196.010457  Topic33   -7.835  -2.6287

[1291 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         8  0.934461     abbott
1        25  0.950271        abc
2         1  0.329076   abortion
2         2  0.658152   abortion
3         4  0.976026     abrams
...     ...       ...        ...
1165      6  0.990019       york
1166     11  0.925543      young
1168     13  0.959492     zeldin
1169     11  0.924669  zelenskyy
1170     25  0.943906     zucker

[903 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[21, 16, 3, 24, 25, 6, 20, 10, 33, 26, 5, 1, 19, 17, 28, 13, 27, 18, 12, 4, 23, 15, 14, 31, 9, 11, 30, 7, 32, 29, 22, 2, 8])

In [112]:
## vectorizer, lda_model, n_words=20
df_topic_keywords = show_sklearn_topics(vectorizer, lda_model)
formatted_topics = format_sklearn_topics(df_topic_keywords)
formatted_topics

d:\workspace\Aletheia\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,covid,republicans,week,party,dems,game,study,big,music,cuomo,west,member,teacher,fan,allegation,schools,live,second,leadership,kanye
Topic 1,nicolle,wallace,step,tom,woke,results,general,senator,lot,set,peace,department,joe,wave,lincoln,husband,raid,surge,big,toyota
Topic 2,house,race,senate,white,death,american,crisis,virginia,theory,critical,claim,governor,board,states,social,battle,south,defense,fentanyl,university
Topic 3,expert,health,book,harry,final,scandal,queen,meghan,prince,force,king,charles,mother,update,criticism,markle,heart,author,elizabeth,today
Topic 4,ukraine,war,russian,voter,president,world,speech,photo,victim,message,drug,economy,use,potential,ukrainian,sister,email,zelenskyy,young,soldier
Topic 5,new,day,york,times,gun,veteran,george,rights,sex,act,teen,administration,effort,great,americas,thousand,abuse,mandate,mexico,fight
Topic 6,withdrawal,aoc,anniversary,instagram,fan,nicole,troop,actor,security,afghanistan,gen,better,book,prosecutor,shia,year,urban,middle,wedding,gender
Topic 7,tesla,reports,samantha,summit,town,brady,analysis,saturday,center,hobbs,different,minor,conservative,oklahoma,education,leaders,coach,decision,freedom,pickup
Topic 8,cnn,reporter,clinton,claims,zucker,nbc,past,narrative,andrew,anchor,abc,boston,hillary,whale,hannahjones,interview,cbs,disinformation,bad,worst
Topic 9,texas,star,student,pelosi,military,political,plan,illegal,michigan,way,loan,immigrant,years,handout,dog,raid,nancy,hospital,abbott,movie


### Write to Pickle File

In [113]:
# create an iterator object with write permission - model.pkl
with open('../pickles/sklearn_count_vectorization_noun_chunks_v1', 'wb') as files:
    pickle.dump(lda_model, files)

## SKLearn with Tri-Grams

In [114]:
## lemmatization with bigrams
# data_words_bigrams = make_bigrams(data_words, bigram_mod)

data_words_trigrams = make_trigrams(data_words, bigram_mod, trigram_mod)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
## performaing lemmatization with noun chunks to preserve the ngram words. 
data_lemmatized = lemmatization_noun_chunks(data_words_trigrams)

# Remove Stop Words
## we are removing stop words in the lemmatization function using spacy is_stop flag
# data_words_nostops = remove_stopwords(data_lemmatized)

print(data_lemmatized[:1])

[['biden', 'xi', 'meeting', 'attempt', 'china', 'taiwan']]


In [115]:
data_lemmatized

[['biden', 'xi', 'meeting', 'attempt', 'china', 'taiwan'],
 ['gop', 'rep', 'calvert', 'election', 'california', 'house', 'seat'],
 ['pelosi', 'future', 'house', 'majority', 'spox'],
 ['arizona',
  'gov',
  'election',
  'katie_hobbs',
  'gop',
  'challenger',
  'kari_lake',
  'race',
  'recount',
  'state',
  'law'],
 ['idaho',
  'student',
  'homicide',
  'crime',
  'passion',
  'burglary',
  'motive',
  'mayor'],
 ['jennifer',
  'siebel',
  'newsom',
  'harvey_weinstein',
  'rapist',
  'sexual_assault',
  'testimony'],
 ['commander', 'eagles', 'loss', 'season', 'wild', 'nfc', 'east', 'upset'],
 ['arizona',
  'gop',
  'rep',
  'schweikert',
  'challenger',
  'republicans',
  'house',
  'majority'],
 ['marjorie',
  'taylor',
  'greene',
  'gop',
  'colleague',
  'mccarthy',
  'speaker',
  'bad',
  'strategy'],
 ['student_loan',
  'buyout',
  'abortion',
  'midterm',
  'democrats',
  'biden',
  'critic'],
 ['hall',
  'famer',
  'tim',
  'hardaway',
  'rape',
  'comment',
  'special',
  

### Count Vectorizer

In [116]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform([" ".join(lem_word) for lem_word in data_lemmatized])

In [117]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")


Sparsicity:  0.4063710841443506 %


### Building Topic Model

In [119]:
# Build LDA Model 35
lda_model = LatentDirichletAllocation(n_components=33,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=33, n_jobs=-1,
                          random_state=100)


### Analyzing Model

In [120]:
analyze_sklearn_lda_model(lda_model, data_vectorized)

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -239448.82377936013
Perplexity:  1170.1017032659263
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 33,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


### Visualizing Topics

In [121]:
dominant_topics = print_sklearn_dominant_topics(lda_model, data_vectorized)
dominant_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,Topic25,Topic26,Topic27,Topic28,Topic29,Topic30,Topic31,Topic32,dominant_topic
Doc0,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.670000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,6
Doc1,0.000000,0.000000,0.000000,0.150000,0.000000,0.000000,0.000000,0.720000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7
Doc2,0.010000,0.410000,0.010000,0.010000,0.010000,0.010000,0.010000,0.410000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,1
Doc3,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.650000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24
Doc4,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.210000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.210000,0.010000,0.210000,0.010000,0.010000,0.010000,0.210000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,7
Doc5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.860000,32
Doc6,0.210000,0.010000,0.010000,0.010000,0.010000,0.010000,0.210000,0.410000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,7
Doc7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.620000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.130000,0.130000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7
Doc8,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.170000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.670000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,17
Doc9,0.000000,0.000000,0.000000,0.130000,0.000000,0.000000,0.130000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.130000,0.130000,0.380000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19


In [122]:
topic_distribution = print_sklearn_topic_distribution(lda_model, data_vectorized)
topic_distribution

,Topic,Num Documents
0,1,733
1,0,719
2,2,642
3,10,515
4,3,423
5,6,407
6,18,378
7,7,370
8,4,310
9,5,297


In [123]:
panel = visualize_sklearn_lda_model(lda_model, data_vectorized, vectorizer)
panel

d:\workspace\Aletheia\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.

d:\workspace\Aletheia\env\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.

d:\workspace\Aletheia\env\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

d:\workspace\Aletheia\env\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



PreparedData(topic_coordinates=               x          y  topics  cluster      Freq
topic                                                 
10    -48.919239  -6.560654       1        1  5.290027
1     -76.166542  -9.485081       2        1  5.143586
18    -42.161819 -30.160267       3        1  5.119954
19    -61.560291  20.161993       4        1  4.395120
7     -20.547855 -73.373222       5        1  4.287313
6      42.657387  54.159672       6        1  4.276563
2     -59.494904  46.049797       7        1  4.231072
31     63.092125  28.903294       8        1  3.993138
22    -32.848267  38.553291       9        1  3.847977
0      47.102688 -55.161568      10        1  3.765510
3      41.606979 -20.812984      11        1  3.682147
24    -34.928986 -52.961620      12        1  3.631491
25    -27.789194  68.657478      13        1  3.311973
16     -9.278261  54.291599      14        1  3.276927
14     -4.074765 -50.408905      15        1  3.256721
21     65.272270 -27.642876      16        1  3.236050
4     -33.235867  14.516580      17        1  3.141900
26     60.697521   0.734041      18        1  3.028033
30     13.447790  37.045918      19        1  3.013138
23    -65.420425 -42.243137      20        1  2.809026
27     16.625397 -67.385681      21        1  2.787877
5      15.111651  65.908096      22        1  2.787014
9      34.946251  26.932556      23        1  2.508664
17     24.585428 -39.293068      24        1  2.317379
8     -17.276979 -28.724840      25        1  2.017943
32     34.247829   4.949896      26        1  1.736686
11      5.365294 -28.092175      27        1  1.695333
28     -8.912347  29.023096      28        1  1.691115
15    -24.295820  -7.494877      29        1  1.495035
13     18.897181 -10.370337      30        1  1.338506
12     11.329117  12.065025      31        1  1.101292
29     -1.645489  -8.162411      32        1  0.932863
20     -9.515964   8.701823      33        1  0.852626, topic_info=         Term        Freq       Total Category  logprob  loglift
90      biden  603.000000  603.000000  Default  30.0000  30.0000
439     house  280.000000  280.000000  Default  29.0000  29.0000
793    report  234.000000  234.000000  Default  28.0000  28.0000
987     trump  256.000000  256.000000  Default  27.0000  27.0000
294  election  241.000000  241.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
286     drone    0.050971   32.780174  Topic33  -8.6429  -1.7017
440     human    0.050971   31.511563  Topic33  -8.6429  -1.6623
393     guard    0.050971   13.425189  Topic33  -8.6429  -0.8090
883    source    0.050971   27.927942  Topic33  -8.6429  -1.5415
521      lake    0.050971   20.779465  Topic33  -8.6429  -1.2459

[1167 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0        12  0.967941     abbott
1        25  0.943479        abc
2         1  0.295540   abortion
2         3  0.703384   abortion
3         3  0.972414     abrams
...     ...       ...        ...
1070     17  0.946731   youngkin
1071     12  0.039435     zeldin
1071     21  0.907012     zeldin
1072      7  0.947387  zelenskyy
1073     15  0.955474     zucker

[950 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 2, 19, 20, 8, 7, 3, 32, 23, 1, 4, 25, 26, 17, 15, 22, 5, 27, 31, 24, 28, 6, 10, 18, 9, 33, 12, 29, 16, 14, 13, 30, 21])

In [124]:
## vectorizer, lda_model, n_words=20
df_topic_keywords = show_sklearn_topics(vectorizer, lda_model)
formatted_topics = format_sklearn_topics(df_topic_keywords)
formatted_topics

d:\workspace\Aletheia\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,police,man,coronavirus,city,case,inflation,americans,study,christmas,team,austin,chicago,number,pandemic,arrest,msnbcs,loss,comment,dozen,girlfriend
Topic 1,border,official,migrant,climate,crisis,pelosi,expert,dem,policy,officer,protest,change,activist,lawmaker,mother,meghan,prince,cancer,illegal,southern
Topic 2,ukraine,war,president,florida,world,deal,veteran,month,nation,message,invasion,battle,national,effort,service,putin,air,americas,soldier,agent
Topic 3,election,midterm,time,bidens,record,money,liberal,mark,group,husband,dark,network,jim,clinton,evidence,cash,prolife,aid,manchin,politic
Topic 4,white,video,campaign,virginia,member,queen,response,king,owner,jan,charles,probe,university,ban,criticism,committee,weapon,sister,heart,performance
Topic 5,report,star,way,concern,oregon,putin,netflix,doc,chinas,regime,center,role,american,hearing,celebrity,summit,williams,doctor,fear,coach
Topic 6,biden,china,fbi,taiwan,admin,cop,source,meeting,kanye,point,win,commander,wave,years,reduction,backlash,matter,red,documentary,dan
Topic 7,house,gop,rep,california,dems,mayor,majority,primary,tim,ohio,ryan,seat,season,handout,kelly,letter,suicide,governor,challenger,work
Topic 8,speech,education,voting,job,drug,department,use,visit,missile,supporter,island,nbc,alaska,rise,reform,cbs,greg,choice,fauci,cartel
Topic 9,woman,child,nyc,support,board,nick,taliban,human,father,relationship,kim,attempt,model,kardashian,cannon,silence,kabul,secret,hate,flag


### Write to Pickle File

In [125]:
# create an iterator object with write permission - model.pkl
with open('../pickles/sklearn_count_vectorization_tri_grams_v1', 'wb') as files:
    pickle.dump(lda_model, files)